### import libraries

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser
from dotenv import load_dotenv
from langchain import HuggingFaceHub
from langchain_google_genai import ChatGoogleGenerativeAI

In [24]:
load_dotenv()

True

### define chain

In [25]:
# define chatbot
chat_model = ChatGoogleGenerativeAI(model="gemini-pro",
                                   temperature=0.3)
# chat_model = HuggingFaceHub(
#     repo_id="google/flan-t5-large",  model_kwargs={"temperature": 0, "max_length": 64})

In [26]:
# define prompt template
prompt_template_text = """You are a high school history teacher grading homework assignments. \
Based on the homework question indicated by “**Q:**” and the correct answer indicated by “**A:**”, your task is to determine whether the student's answer is correct. \
Grading is binary; therefore, student answers can be correct or wrong. \
Simple misspellings are okay.

**Q:** {question}
**A:** {correct_answer}

**Student's Answer:** {student_answer}
"""

prompt = PromptTemplate(input_variables=["question", "correct_answer", "student_answer"], template = prompt_template_text)

In [27]:
# define chain
chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
)

### use chain

In [28]:
# define inputs
question = "Who was the 35th president of the United States of America?"
correct_answer = "John F. Kennedy"
student_answer =  "JFK"

# run chain
chain.run({'question':question, 'correct_answer':correct_answer, 'student_answer':student_answer})

'Correct'

In [29]:
# run chain in for loop
student_answer_list = ["John F. Kennedy", "Mayur", "FDR", "John F. Kenedy", "John Kennedy", "Jack Kennedy", "Jacqueline Kennedy", "Robert F. Kenedy"]

for student_answer in student_answer_list:
    print(student_answer + " - " + str(chain.run({'question':question, 'correct_answer':correct_answer, 'student_answer':student_answer})))

John F. Kennedy - Correct
Mayur - Wrong
FDR - Wrong
John F. Kenedy - Correct
John Kennedy - Correct
Jack Kennedy - Correct
Jacqueline Kennedy - Wrong
Robert F. Kenedy - Wrong


### add output parser

In [33]:
# define output parser
class GradeOutputParser(BaseOutputParser):
    """Determine whether grade was correct or wrong"""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return "wrong" not in text.lower()

In [34]:
# update chain
chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    output_parser=GradeOutputParser()
)

In [35]:
# grade student answers
for student_answer in student_answer_list:
    print(student_answer + " - " + str(chain.run({'question':question, 'correct_answer':correct_answer, 'student_answer':student_answer})))

John F. Kennedy - True
Mayur - False
FDR - False
John F. Kenedy - False
John Kennedy - True
Jack Kennedy - True
Jacqueline Kennedy - False
Robert F. Kenedy - False
